# Readmission Prediction

This notebook showcases readmission prediction on the [Diabetes 130-US Hospitals for Years 1999-2008](https://archive.ics.uci.edu/dataset/296/diabetes+130-us+hospitals+for+years+1999-2008) using CyclOps. The task is formulated as a binary classification task, where we predict the probability of early readmission of the patient within 30 days of discharge.

## Install libraries

In [ ]:
!pip install pycyclops
!pip install ucimlrepo

## Import Libraries

In [ ]:
"""Readmission prediction."""

# ruff: noqa: E402

import copy
import inspect
from datetime import date

import numpy as np
import pandas as pd
import plotly.express as px
from datasets import Dataset
from datasets.features import ClassLabel
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from ucimlrepo import fetch_ucirepo

from cyclops.data.df.feature import TabularFeatures
from cyclops.data.slicer import SliceSpec
from cyclops.evaluate.fairness import FairnessConfig  # noqa: E402
from cyclops.evaluate.metrics import create_metric
from cyclops.evaluate.metrics.experimental.functional import (
    binary_npv,
    binary_ppv,
    binary_roc,
)
from cyclops.evaluate.metrics.experimental.metric_dict import MetricDict
from cyclops.models.catalog import create_model
from cyclops.report import ModelCardReport
from cyclops.report.plot.classification import ClassificationPlotter
from cyclops.report.utils import flatten_results_dict
from cyclops.tasks import BinaryTabularClassificationTask

CyclOps offers a package for documentation of the model through a model report. The `ModelCardReport` class is used to populate and generate the model report as an HTML file. The model report has the following sections:

- Overview: Provides a high level overview of how the model is doing (a quick glance of important metrics), and how it is doing over time (performance over several metrics and subgroups over time).
- Datasets: High level statistics of the training data, including changes in distribution over time.
- Quantitative Analysis: This section contains additional detailed performance metrics of the model for different sets of the data and subpopulations.
- Fairness Analysis: This section contains the fairness metrics of the model.
- Model Details: This section contains descriptive metadata about the model such as the owners, version, license, etc.
- Model Parameters: This section contains the technical details of the model such as the model architecture, training parameters, etc.
- Considerations: This section contains descriptions of the considerations involved in developing and using the model such as the intended use, limitations, etc.

We will use this to document the model development process as we go along and generate the model report at the end.

`The model report tool is a work in progress and is subject to change.`

In [ ]:
report = ModelCardReport()

## Constants

In [ ]:
RANDOM_SEED = 85
NAN_THRESHOLD = 0.75
TRAIN_SIZE = 0.8
EVAL_NUM = 3

## Data Loading

In [ ]:
diabetes_130_data = fetch_ucirepo(id=296)
features = diabetes_130_data["data"]["features"]
targets = diabetes_130_data["data"]["targets"]
metadata = diabetes_130_data["metadata"]
variables = diabetes_130_data["variables"]

In [ ]:
metadata

In [ ]:
def transform_label(value):
    """Transform string labels of readmission into 0/1 binary labels.

    Parameters
    ----------
    value: str
        Input value

    Returns
    -------
    int
        0 if not readmitted or if greater than 30 days, 1 if less than 30 days

    """
    if value in ["NO", ">30"]:
        return 0
    if value == "<30":
        return 1

    raise ValueError("Unexpected value for readmission!")


df = features
targets["readmitted"] = targets["readmitted"].apply(transform_label)
df["readmitted"] = targets

Choose a small subset for modelling

In [ ]:
df = df[0:1000]

Remove features that are NaNs or have just a single unique value

In [ ]:
features_to_remove = []
for col in df:
    if len(df[col].value_counts()) <= 1:
        features_to_remove.append(col)
df = df.drop(columns=features_to_remove)

### Sex values

In [ ]:
fig = px.pie(df, names="gender")

fig.update_layout(
    title="Gender Distribution",
)

fig.show()

**Add the figure to the report**

In [ ]:
report.log_plotly_figure(
    fig=fig,
    caption="Gender Distribution",
    section_name="datasets",
)

###  Age distribution

In [ ]:
fig = px.histogram(df, x="age")
fig.update_layout(
    title="Age Distribution",
    xaxis_title="Age",
    yaxis_title="Count",
    bargap=0.2,
)

fig.show()

**Add the figure to the report**

In [ ]:
report.log_plotly_figure(
    fig=fig,
    caption="Age Distribution",
    section_name="datasets",
)

### Outcome distribution

In [ ]:
df["outcome"] = df["readmitted"].astype("int")
df = df.drop(columns=["readmitted"])

In [ ]:
fig = px.pie(df, names="outcome")
fig.update_traces(textinfo="percent+label")
fig.update_layout(title_text="Outcome Distribution")
fig.update_traces(
    hovertemplate="Outcome: %{label}<br>Count: \
    %{value}<br>Percent: %{percent}",
)
fig.show()

**Add the figure to the report**

In [ ]:
report.log_plotly_figure(
    fig=fig,
    caption="Outcome Distribution",
    section_name="datasets",
)

In [ ]:
class_counts = df["outcome"].value_counts()
class_ratio = class_counts[0] / class_counts[1]
print(class_ratio, class_counts)

From the features in the dataset, we select all of them to train the model!

In [ ]:
features_list = list(df.columns)
features_list.remove("outcome")
features_list = sorted(features_list)

### Identifying feature types

Cyclops `TabularFeatures` class helps to identify feature types, an essential step before preprocessing the data. Understanding feature types (numerical/categorical/binary) allows us to apply appropriate preprocessing steps for each type.

In [ ]:
tab_features = TabularFeatures(
    data=df.reset_index(),
    features=features_list,
    by="index",
    targets="outcome",
)
print(tab_features.types)

### Creating data preprocessors

We create a data preprocessor using sklearn's ColumnTransformer. This helps in applying different preprocessing steps to different columns in the dataframe. For instance, binary features might be processed differently from numeric features.

In [ ]:
numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", MinMaxScaler())],
)

binary_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent"))],
)

In [ ]:
numeric_features = sorted((tab_features.features_by_type("numeric")))
numeric_indices = [
    df[features_list].columns.get_loc(column) for column in numeric_features
]
print(numeric_features)

In [ ]:
binary_features = sorted(tab_features.features_by_type("binary"))
binary_features.remove("outcome")
ordinal_features = sorted(
    tab_features.features_by_type("ordinal")
    + ["medical_specialty", "diag_1", "diag_2", "diag_3"]
)
binary_indices = [
    df[features_list].columns.get_loc(column) for column in binary_features
]
ordinal_indices = [
    df[features_list].columns.get_loc(column) for column in ordinal_features
]
print(binary_features, ordinal_features)

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_indices),
        (
            "onehot",
            OneHotEncoder(handle_unknown="ignore", sparse_output=False),
            binary_indices + ordinal_indices,
        ),
    ],
    remainder="passthrough",
)

Let's document the dataset in the model card. This can be done using the `log_dataset` method, which takes the following arguments:
- description: A description of the dataset.
- citation: The citation for the dataset.
- link: A link to a resource for the dataset.
- license_id: The SPDX license identifier for the dataset.
- version: The version of the dataset.
- features: A list of features in the dataset.
- split: The split of the dataset (train, test, validation, etc.).
- sensitive_features: A list of sensitive features used to train/evaluate the model.
- sensitive_feature_justification: A justification for the sensitive features used to train/evaluate the model.

In [ ]:
report.log_dataset(
    description=metadata["abstract"],
    citation=inspect.cleandoc(
        """
        @article{strack2014impact,
          title={Impact of HbA1c measurement on hospital readmission rates: analysis of 70,000 clinical database patient records},
          author={Strack, Beata and DeShazo, Jonathan P and Gennings, Chris and Olmo, Juan L and Ventura, Sebastian and Cios, Krzysztof J and Clore, John N and others},
          journal={BioMed research international},
          volume={2014},
          year={2014},
          publisher={Hindawi}
        }
    """,
    ),
    link=metadata["repository_url"],
    license_id="CC0-1.0",
    version="Version 1",
    features=features_list,
    sensitive_features=["gender", "age", "race"],
    sensitive_feature_justification="Demographic information like age and gender \
        often have a strong correlation with health outcomes. For example, older \
        patients are more likely to have a higher risk of readmission.",
)

## Creating Hugging Face Dataset

We convert our processed Pandas dataframe into a Hugging Face dataset, a powerful and easy-to-use data format which is also compatible with CyclOps models and evaluator modules. The dataset is then split to train and test sets.

In [ ]:
dataset = Dataset.from_pandas(df)
dataset.cleanup_cache_files()
print(dataset)

In [ ]:
dataset = dataset.cast_column("outcome", ClassLabel(num_classes=2))
dataset = dataset.train_test_split(
    train_size=TRAIN_SIZE,
    stratify_by_column="outcome",
    seed=RANDOM_SEED,
)

## Model Creation

CyclOps model registry allows for straightforward creation and selection of models. This registry maintains a list of pre-configured models, which can be instantiated with a single line of code. Here we use a SGD classifier to fit a logisitic regression model. The model configurations can be passed to `create_model` based on the sklearn parameters for SGDClassifier.

In [ ]:
model_name = "xgb_classifier"
model = create_model(model_name, random_state=123)

## Task Creation

We use Cyclops tasks to define our model's task (in this case, readmission prediction), train the model, make predictions, and evaluate performance. Cyclops task classes encapsulate the entire ML pipeline into a single, cohesive structure, making the process smooth and easy to manage.

In [ ]:
readmission_prediction_task = BinaryTabularClassificationTask(
    {model_name: model},
    task_features=features_list,
    task_target="outcome",
)

In [ ]:
readmission_prediction_task.list_models()

## Training

If `best_model_params` is passed to the `train` method, the best model will be selected after the hyperparameter search. The parameters in `best_model_params` indicate the values to create the parameters grid.

Note that the data preprocessor needs to be passed to the tasks methods if the Hugging Face dataset is not already preprocessed. 

In [ ]:
best_model_params = {
    "n_estimators": [100, 250, 500],
    "learning_rate": [0.1, 0.01],
    "max_depth": [2, 5],
    "reg_lambda": [0, 1, 10],
    "colsample_bytree": [0.7, 0.8, 1],
    "gamma": [0, 1, 2, 10],
    "method": "random",
    "scale_pos_weight": [int(class_ratio)],
}
readmission_prediction_task.train(
    dataset["train"],
    model_name=model_name,
    transforms=preprocessor,
    best_model_params=best_model_params,
)

In [ ]:
model_params = readmission_prediction_task.list_models_params()[model_name]
print(model_params)

**Log the model parameters to the report.**

We can add model parameters to the model card using the `log_model_parameters` method.

In [ ]:
report.log_model_parameters(params=model_params)

## Prediction

The prediction output can be either the whole Hugging Face dataset with the prediction columns added to it or the single column containing the predicted values.

In [ ]:
y_pred = readmission_prediction_task.predict(
    dataset["test"],
    model_name=model_name,
    transforms=preprocessor,
    proba=True,
    only_predictions=True,
)
prediction_df = pd.DataFrame(
    {
        "y_prob": [y_pred_i[1] for y_pred_i in y_pred],
        "y_true": dataset["test"]["outcome"],
        "gender": dataset["test"]["gender"],
    }
)

## Evaluation

Evaluation is done using various evaluation metrics that provide different perspectives on the model's predictive abilities i.e. standard performance metrics and fairness metrics.

The standard performance metrics can be created using the `MetricDict` object.

In [ ]:
metric_names = [
    "binary_accuracy",
    "binary_precision",
    "binary_recall",
    "binary_f1_score",
    "binary_auroc",
    "binary_average_precision",
    "binary_roc_curve",
    "binary_precision_recall_curve",
]
metrics = [
    create_metric(metric_name, experimental=True) for metric_name in metric_names
]
metric_collection = MetricDict(metrics)

In addition to overall metrics, it might be interesting to see how the model performs on certain subpopulations. We can define these subpopulations using `SliceSpec` objects. 

In [ ]:
spec_list = [
    {
        "age": {
            "value": "[50-60)",
        },
    },
    {
        "age": {
            "value": "[60-70)",
        },
    },
]
slice_spec = SliceSpec(spec_list)

A `MetricDict` can also be defined for the fairness metrics.

In [ ]:
specificity = create_metric(metric_name="binary_specificity", experimental=True)
sensitivity = create_metric(metric_name="binary_sensitivity", experimental=True)

fpr = -specificity + 1
fnr = -sensitivity + 1

ber = (fpr + fnr) / 2

fairness_metric_collection = MetricDict(
    {
        "Sensitivity": sensitivity,
        "Specificity": specificity,
        "BER": ber,
    },
)

The FairnessConfig helps in setting up and evaluating the fairness of the model predictions.

In [ ]:
fairness_config = FairnessConfig(
    metrics=fairness_metric_collection,
    dataset=None,  # dataset is passed from the evaluator
    target_columns=None,  # target columns are passed from the evaluator
    groups=["age"],
    group_base_values={"age": "[40-50)"},
    thresholds=[0.5],
)

The evaluate methods outputs the evaluation results and the Hugging Face dataset with the predictions added to it.

In [ ]:
results, dataset_with_preds = readmission_prediction_task.evaluate(
    dataset=dataset["test"],
    metrics=metric_collection,
    model_names=model_name,
    transforms=preprocessor,
    prediction_column_prefix="preds",
    slice_spec=slice_spec,
    batch_size=-1,
    fairness_config=fairness_config,
    override_fairness_metrics=False,
)

In [ ]:
results_female, _ = readmission_prediction_task.evaluate(
    dataset=dataset["test"],
    metrics=MetricDict(
        {
            "BinaryAccuracy": create_metric(
                metric_name="binary_accuracy",
                experimental=True,
            ),
        },
    ),
    model_names=model_name,
    transforms=preprocessor,
    prediction_column_prefix="preds",
    slice_spec=SliceSpec([{"gender": {"value": "Female"}}], include_overall=False),
    batch_size=-1,
)

**Log the performance metrics to the report.**

We can add a performance metric to the model card using the `log_performance_metric` method, which expects a dictionary where the keys are in the following format: `slice_name/metric_name`. For instance, `overall/accuracy`. 

We first need to process the evaluation results to get the metrics in the right format.

In [ ]:
model_name = f"model_for_preds.{model_name}"
results_flat = flatten_results_dict(
    results=results,
    remove_metrics=["BinaryROC", "BinaryPrecisionRecallCurve"],
    model_name=model_name,
)
results_female_flat = flatten_results_dict(
    results=results_female,
    model_name=model_name,
)
# ruff: noqa: W505
for name, metric in results_female_flat.items():
    split, name = name.split("/")  # noqa: PLW2901
    descriptions = {
        "BinaryPrecision": "The proportion of predicted positive instances that are correctly predicted.",
        "BinaryRecall": "The proportion of actual positive instances that are correctly predicted. Also known as recall or true positive rate.",
        "BinaryAccuracy": "The proportion of all instances that are correctly predicted.",
        "BinaryAUROC": "The area under the receiver operating characteristic curve (AUROC) is a measure of the performance of a binary classification model.",
        "BinaryAveragePrecision": "The area under the precision-recall curve (AUPRC) is a measure of the performance of a binary classification model.",
        "BinaryF1Score": "The harmonic mean of precision and recall.",
    }
    report.log_quantitative_analysis(
        "performance",
        name=name,
        value=metric.tolist(),
        description=descriptions[name],
        metric_slice=split,
        pass_fail_thresholds=0.7,
        pass_fail_threshold_fns=lambda x, threshold: bool(x >= threshold),
    )

for name, metric in results_flat.items():
    split, name = name.split("/")  # noqa: PLW2901
    descriptions = {
        "BinaryPrecision": "The proportion of predicted positive instances that are correctly predicted.",
        "BinaryRecall": "The proportion of actual positive instances that are correctly predicted. Also known as recall or true positive rate.",
        "BinaryAccuracy": "The proportion of all instances that are correctly predicted.",
        "BinaryAUROC": "The area under the receiver operating characteristic curve (AUROC) is a measure of the performance of a binary classification model.",
        "BinaryAveragePrecision": "The area under the precision-recall curve (AUPRC) is a measure of the performance of a binary classification model.",
        "BinaryF1Score": "The harmonic mean of precision and recall.",
    }
    report.log_quantitative_analysis(
        "performance",
        name=name,
        value=metric.tolist(),
        description=descriptions[name],
        metric_slice=split,
        pass_fail_thresholds=0.7,
        pass_fail_threshold_fns=lambda x, threshold: bool(x >= threshold),
    )

We can also use the `ClassificationPlotter` to plot the performance metrics and the add the figure to the model card using the `log_plotly_figure` method.

In [ ]:
plotter = ClassificationPlotter(task_type="binary", class_names=["0", "1"])
plotter.set_template("plotly_white")

In [ ]:
# extracting the ROC curves and AUROC results for all the slices
roc_curves = {
    slice_name: slice_results["BinaryROC"]
    for slice_name, slice_results in results[model_name].items()
}
aurocs = {
    slice_name: slice_results["BinaryAUROC"]
    for slice_name, slice_results in results[model_name].items()
}
roc_curves.keys()

In [ ]:
# plotting the ROC curves for all the slices
roc_plot = plotter.roc_curve_comparison(roc_curves, aurocs=aurocs)
report.log_plotly_figure(
    fig=roc_plot,
    caption="ROC Curve for Female Patients",
    section_name="quantitative analysis",
)
roc_plot.show()

In [ ]:
# extracting the precision-recall curves and average precision results for all the slices
pr_curves = {
    slice_name: slice_results["BinaryPrecisionRecallCurve"]
    for slice_name, slice_results in results[model_name].items()
}
average_precisions = {
    slice_name: slice_results["BinaryAveragePrecision"]
    for slice_name, slice_results in results[model_name].items()
}
pr_curves.keys()

In [ ]:
# plotting the precision-recall curves for all the slices
pr_plot = plotter.precision_recall_curve_comparison(
    pr_curves,
    auprcs=average_precisions,
)
report.log_plotly_figure(
    fig=pr_plot,
    caption="Precision-Recall Curve Comparison",
    section_name="quantitative analysis",
)
pr_plot.show()

In [ ]:
# Extracting the overall classification metric values.
overall_performance = {
    metric_name: metric_value
    for metric_name, metric_value in results[model_name]["overall"].items()
    if metric_name not in ["BinaryROC", "BinaryPrecisionRecallCurve"]
}

In [ ]:
# Plotting the overall classification metric values.
overall_performance_plot = plotter.metrics_value(
    overall_performance,
    title="Overall Performance",
)
report.log_plotly_figure(
    fig=overall_performance_plot,
    caption="Overall Performance",
    section_name="quantitative analysis",
)
overall_performance_plot.show()

In [ ]:
# Extracting the metric values for all the slices.
slice_metrics = {
    slice_name: {
        metric_name: metric_value
        for metric_name, metric_value in slice_results.items()
        if metric_name not in ["BinaryROC", "BinaryPrecisionRecallCurve"]
    }
    for slice_name, slice_results in results[model_name].items()
}

In [ ]:
# Plotting the metric values for all the slices.
slice_metrics_plot = plotter.metrics_comparison_bar(slice_metrics)
report.log_plotly_figure(
    fig=slice_metrics_plot,
    caption="Slice Metric Comparison",
    section_name="quantitative analysis",
)
slice_metrics_plot.show()

In [ ]:
# Plotting the metric values for all the slices.
# ROC curve components
pred_probs = np.array(dataset_with_preds["preds.xgb_classifier"])
true_labels = np.array(dataset_with_preds["outcome"])
roc_curve = binary_roc(true_labels, pred_probs)
ppv = np.zeros_like(roc_curve.thresholds)
npv = np.zeros_like(roc_curve.thresholds)

# Calculate PPV and NPV for each threshold
for i, threshold in enumerate(roc_curve.thresholds):
    # Calculate PPV and NPV
    ppv[i] = binary_ppv(true_labels, pred_probs, threshold=threshold)
    npv[i] = binary_npv(true_labels, pred_probs, threshold=threshold)
runway_plot = plotter.threshperf(roc_curve, ppv, npv, pred_probs)
report.log_plotly_figure(
    fig=runway_plot,
    caption="Threshold-Performance plot",
    section_name="quantitative analysis",
)
runway_plot.show()

We can also plot the calibration curve of the model on the test set

In [ ]:
calibration_plot = plotter.calibration(
    prediction_df, y_true_col="y_true", y_prob_col="y_prob", group_col="gender"
)
report.log_plotly_figure(
    fig=calibration_plot,
    caption="Calibration plot",
    section_name="quantitative analysis",
)
calibration_plot.show()

In [ ]:
# Reformatting the fairness metrics
fairness_results = copy.deepcopy(results["fairness"])
fairness_metrics = {}
# remove the group size from the fairness results and add it to the slice name
for slice_name, slice_results in fairness_results.items():
    group_size = slice_results.pop("Group Size")
    fairness_metrics[f"{slice_name} (Size={group_size})"] = slice_results

In [ ]:
# Plotting the fairness metrics
fairness_plot = plotter.metrics_comparison_scatter(
    fairness_metrics,
    title="Fairness Metrics",
)
report.log_plotly_figure(
    fig=fairness_plot,
    caption="Fairness Metrics",
    section_name="fairness analysis",
)
fairness_plot.show()

## Report Generation

Before generating the model card, let us document some of the details of the model and some considerations involved in developing and using the model.


Let's start with populating the model details section, which includes the following fields by default:
- description: A high-level description of the model and its usage for a general audience.
- version: The version of the model.
- owners: The individuals or organizations that own the model.
- license: The license under which the model is made available.
- citation: The citation for the model.
- references: Links to resources that are relevant to the model.
- path: The path to where the model is stored.
- regulatory_requirements: The regulatory requirements that are relevant to the model.

We can add additional fields to the model details section by passing a dictionary to the `log_from_dict` method and specifying the section name as `model_details`. You can also use the `log_descriptor` method to add a new field object with a `description` attribute to any section of the model card.

In [ ]:
report.log_from_dict(
    data={
        "name": "Readmission Prediction Model",
        "description": "The model was trained on the Diabetes 130-US Hospitals for Years 1999-2008 \
        dataset to predict risk of readmission within 30 days of discharge.",
    },
    section_name="model_details",
)

report.log_version(
    version_str="0.0.1",
    date=str(date.today()),
    description="Initial Release",
)
report.log_owner(
    name="CyclOps Team",
    contact="vectorinstitute.github.io/cyclops/",
    email="cyclops@vectorinstitute.ai",
)
report.log_license(identifier="Apache-2.0")
report.log_reference(
    link="https://xgboost.readthedocs.io/en/stable/python/python_api.html",  # noqa: E501
)

Next, let's populate the considerations section, which includes the following fields by default:
- users: The intended users of the model.
- use_cases: The use cases for the model. These could be primary, downstream or out-of-scope use cases.
- fairness_assessment: A description of the benefits and harms of the model for different groups as well as the steps taken to mitigate the harms.
- ethical_considerations: The risks associated with using the model and the steps taken to mitigate them. This can be populated using the  `log_risk` method.



In [ ]:
report.log_from_dict(
    data={
        "users": [
            {"description": "Hospitals"},
            {"description": "Clinicians"},
        ],
    },
    section_name="considerations",
)
report.log_user(description="ML Engineers")
report.log_use_case(
    description="Predicting risk of readmission.",
    kind="primary",
)
report.log_use_case(
    description="Predicting risk of pathologies and conditions other\
    than risk of readmission.",
    kind="out-of-scope",
)
report.log_fairness_assessment(
    affected_group="sex, age",
    benefit="Improved health outcomes for patients.",
    harm="Biased predictions for patients in certain groups (e.g. older patients) \
        may lead to worse health outcomes.",
    mitigation_strategy="We will monitor the performance of the model on these groups \
        and retrain the model if the performance drops below a certain threshold.",
)
report.log_risk(
    risk="The model may be used to make decisions that affect the health of patients.",
    mitigation_strategy="The model should be continuously monitored for performance \
        and retrained if the performance drops below a certain threshold.",
)

Once the model card is populated, you can generate the report using the `export` method. The report is generated in the form of an HTML file. A JSON file containing the model card data will also be generated along with the HTML file. By default, the files will be saved in a folder named `cyclops_reports` in the current working directory. You can change the path by passing a `output_dir` argument when instantiating the `ModelCardReport` class.

In [ ]:
synthetic_timestamps = pd.date_range(
    start="20/6/2024", periods=3, freq="W"
).values.astype(str)
report_path = report.export(
    output_filename="readmission_report_periodic.html",
    synthetic_timestamp=synthetic_timestamps[EVAL_NUM - 1],
    last_n_evals=3,
)

You can view the generated HTML [report](./heart_failure_report_periodic.html).